In [1]:
import os


In [5]:

data_dir = './dataset'

for x in range(10):
    files = os.listdir(data_dir + "/" + str(x))
    train_files = files[:4000]
    train_files_new = ["./dataset/train/" + str(x) + "/" + e for e in train_files]
    test_files = files[4000:5000]
    test_files_new = ["./dataset/test/" + str(x) + "/" + e for e in test_files]
    